In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/facebook/698.featnames
/kaggle/input/facebook/3980.featnames
/kaggle/input/facebook/686.egofeat
/kaggle/input/facebook/3437.circles
/kaggle/input/facebook/1912.featnames
/kaggle/input/facebook/1684.featnames
/kaggle/input/facebook/1684.egofeat
/kaggle/input/facebook/3437.feat
/kaggle/input/facebook/3437.featnames
/kaggle/input/facebook/686.circles
/kaggle/input/facebook/107.featnames
/kaggle/input/facebook/348.egofeat
/kaggle/input/facebook/0.feat
/kaggle/input/facebook/698.feat
/kaggle/input/facebook/686.feat
/kaggle/input/facebook/107.feat
/kaggle/input/facebook/348.feat
/kaggle/input/facebook/698.circles
/kaggle/input/facebook/0.circles
/kaggle/input/facebook/414.edges
/kaggle/input/facebook/414.feat
/kaggle/input/facebook/1684.feat
/kaggle/input/facebook/698.egofeat
/kaggle/input/facebook/414.circles
/kaggle/input/facebook/1912.edges
/kaggle/input/facebook/1684.edges
/kaggle/input/facebook/686.featnames
/kaggle/input/facebook/107.circles
/kaggle/input/facebook/1912.fe

Files:

nodeId.edges : The edges in the ego network for the node 'nodeId'. Edges are undirected for facebook, and directed (a follows b) for twitter and gplus. The 'ego' node does not appear, but it is assumed that they follow every node id that appears in this file.

nodeId.circles : The set of circles for the ego node. Each line contains one circle, consisting of a series of node ids. The first entry in each line is the name of the circle.

nodeId.feat : The features for each of the nodes that appears in the edge file.

nodeId.egofeat : The features for the ego user.

nodeId.featnames : The names of each of the feature dimensions. Features are '1' if the user has this property in their profile, and '0' otherwise. This file has been anonymized for facebook users, since the names of the features would reveal private data.